In [ ]:
!pip install pandas

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/fitbit")
path = '/root/.cache/kagglehub/datasets/arashnic/fitbit/versions/2/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16'
print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import os

# Use path returned by kagglehub
daily_activity = pd.read_csv(os.path.join(path, "dailyActivity_merged.csv"))
heartrate = pd.read_csv(os.path.join(path, "heartrate_seconds_merged.csv"))
hourly_calories = pd.read_csv(os.path.join(path, "hourlyCalories_merged.csv"))
hourly_intensities = pd.read_csv(os.path.join(path, "hourlyIntensities_merged.csv"))
hourly_steps = pd.read_csv(os.path.join(path, "hourlySteps_merged.csv"))
minute_calories = pd.read_csv(os.path.join(path, "minuteCaloriesNarrow_merged.csv"))
minute_intensities = pd.read_csv(os.path.join(path, "minuteIntensitiesNarrow_merged.csv"))
minute_mets = pd.read_csv(os.path.join(path, "minuteMETsNarrow_merged.csv"))
minute_sleep = pd.read_csv(os.path.join(path, "minuteSleep_merged.csv"))
minute_steps = pd.read_csv(os.path.join(path, "minuteStepsNarrow_merged.csv"))
weight_log = pd.read_csv(os.path.join(path, "weightLogInfo_merged.csv"))


In [ ]:
user_id = 1503960366
user_tables = {
    'Daily Activity': daily_activity[daily_activity['Id'] == user_id],
    'Heart Rate': heartrate[heartrate['Id'] == user_id],
    'Hourly Calories': hourly_calories[hourly_calories['Id'] == user_id],
    'Hourly Intensities': hourly_intensities[hourly_intensities['Id'] == user_id],
    'Hourly Steps': hourly_steps[hourly_steps['Id'] == user_id],
    'Minute Calories': minute_calories[minute_calories['Id'] == user_id],
    'Minute Intensities': minute_intensities[minute_intensities['Id'] == user_id],
    'Minute METs': minute_mets[minute_mets['Id'] == user_id],
    'Minute Sleep': minute_sleep[minute_sleep['Id'] == user_id],
    'Minute Steps': minute_steps[minute_steps['Id'] == user_id],
    'Weight Log': weight_log[weight_log['Id'] == user_id],
}


In [ ]:
for name, df in user_tables.items():
    print(f"\n=== {name} ===")

    # Check if 'StepTotal' column exists in the current DataFrame
    if 'StepTotal' in df.columns:
        filtered_df = df[df['StepTotal'] != 0]
    else:
        filtered_df = df  # no filtering if no StepTotal column

    display(filtered_df)

## Steps analysis day/hrs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

# Step 1: Extract the hourly steps DataFrame for the user
hourly_steps_df = user_tables['Hourly Steps'].copy()

# Step 2: Convert 'ActivityHour' to datetime
hourly_steps_df['ActivityHour'] = pd.to_datetime(hourly_steps_df['ActivityHour'])

# Step 3: Extract 'Date' and 'Hour' from datetime
hourly_steps_df['Date'] = hourly_steps_df['ActivityHour'].dt.date
hourly_steps_df['Hour'] = hourly_steps_df['ActivityHour'].dt.hour

# Step 4: Filter the data for the target date
target_date = date(2016, 3, 25)
filtered_df = hourly_steps_df[hourly_steps_df['Date'] == target_date]

# Step 5: Define time block labels based on hour
def get_time_block(hour):
    if 0 <= hour < 6:
        return '00-06'
    elif 6 <= hour < 12:
        return '06-12'
    elif 12 <= hour < 18:
        return '12-18'
    else:
        return '18-24'

# Step 6: Create a new column for time blocks
filtered_df['TimeBlock'] = filtered_df['Hour'].apply(get_time_block)

# Step 7: Group by TimeBlock and sum steps
steps_by_block = filtered_df.groupby('TimeBlock')['StepTotal'].sum().reindex(['00-06', '06-12', '12-18', '18-24'])

# Step 8: Plot the result
plt.figure(figsize=(8, 5))
steps_by_block.plot(kind='bar', color='mediumseagreen')

plt.title(f'Steps per Time Block on {target_date} for User {user_id}')
plt.xlabel('Time Block')
plt.ylabel('Total Steps')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


total_daily_steps = filtered_df['StepTotal'].sum()
print(f"Total steps on {target_date}: {total_daily_steps}")


## Sleep Analysis minutes/type

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

# Step 1: Get the user's minute-level sleep data
minute_sleep_df = user_tables['Minute Sleep'].copy()

# Step 2: Convert the 'date' column to datetime if it's not already
minute_sleep_df['date'] = pd.to_datetime(minute_sleep_df['date'])

# Step 3: Filter for a specific date
target_date = date(2016, 3, 25)  # Change as needed
filtered_sleep = minute_sleep_df[minute_sleep_df['date'].dt.date == target_date]

# Step 4: Map the numerical 'value' codes to sleep stage names
sleep_stage_map = {
    0: 'Awake',
    1: 'Light',
    2: 'Deep',
    3: 'REM'
}
filtered_sleep['SleepStage'] = filtered_sleep['value'].map(sleep_stage_map)

# Step 5: Count the number of minutes in each sleep stage
sleep_stage_counts = filtered_sleep['SleepStage'].value_counts().reindex(['Awake', 'Light', 'Deep', 'REM'], fill_value=0)

# Step 6: Plot the result
plt.figure(figsize=(8, 5))
sleep_stage_counts.plot(kind='bar', color='slateblue')

plt.title(f'Sleep Stages on {target_date} for User {user_id}')
plt.xlabel('Sleep Stage')
plt.ylabel('Minutes Spent')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


## activities

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

# Step 1: Get the daily activity data for the user
daily_activity_df = user_tables['Daily Activity'].copy()

# Step 2: Convert the date column if it's not datetime
daily_activity_df['ActivityDate'] = pd.to_datetime(daily_activity_df['ActivityDate'])

# Step 3: Filter for the target date
target_date = date(2016, 3, 25)
filtered_df = daily_activity_df[daily_activity_df['ActivityDate'].dt.date == target_date]

# Step 4: Extract activity minutes if the date is found
if not filtered_df.empty:
    light = filtered_df['LightlyActiveMinutes'].iloc[0]
    moderate = filtered_df['FairlyActiveMinutes'].iloc[0]
    active = filtered_df['VeryActiveMinutes'].iloc[0]

    zone_minutes = moderate * 1 + active * 2
    print(f"Zone Minutes on {target_date}: {zone_minutes}")

    # Step 5: Prepare data for plotting
    activity_levels = ['Light', 'Moderate', 'Very Active']
    minutes = [light, moderate, active]

    # Step 6: Plot
    plt.figure(figsize=(7, 5))
    plt.bar(activity_levels, minutes, color=['lightblue', 'orange', 'crimson'])

    plt.title(f'Activity Minutes on {target_date} for User {user_id}')
    plt.xlabel('Activity Intensity')
    plt.ylabel('Minutes Spent')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

else:
    print(f"No data available for {target_date}")


## heart rate/ hrs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

# Step 1: Extract heart rate data
heartrate_df = user_tables['Heart Rate'].copy()

# Step 2: Convert time column to datetime
heartrate_df['Time'] = pd.to_datetime(heartrate_df['Time'])

# Step 3: Filter for target date
target_date = date(2016, 4, 1)
heartrate_df['Date'] = heartrate_df['Time'].dt.date
filtered_df = heartrate_df[heartrate_df['Date'] == target_date]

# Step 4: Extract hour from time
filtered_df['Hour'] = filtered_df['Time'].dt.hour

# Step 5: Define time blocks
def get_time_block(hour):
    if 0 <= hour < 6:
        return '00-06'
    elif 6 <= hour < 12:
        return '06-12'
    elif 12 <= hour < 18:
        return '12-18'
    else:
        return '18-24'

filtered_df['TimeBlock'] = filtered_df['Hour'].apply(get_time_block)

# Step 6: Group by TimeBlock and calculate average heart rate
avg_hr_by_block = filtered_df.groupby('TimeBlock')['Value'].mean().reindex(['00-06', '06-12', '12-18', '18-24'])

# Step 7: Plot
plt.figure(figsize=(8, 5))
avg_hr_by_block.plot(kind='bar', color='tomato')

plt.title(f'Average Heart Rate by Time Block on {target_date} for User {user_id}')
plt.xlabel('Time Block')
plt.ylabel('Avg Heart Rate (bpm)')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


## totalsteps/week

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Copy and prepare the data
daily_df = user_tables['Daily Activity'].copy()
daily_df['ActivityDate'] = pd.to_datetime(daily_df['ActivityDate'])

# Filter for the week of interest: Monday to Sunday
# Example week: Monday 2023-06-08 to Sunday 2023-06-14
start_date = pd.to_datetime('2016-03-26')  # Monday
end_date = pd.to_datetime('2016-04-01')    # Sunday

mask = (daily_df['ActivityDate'] >= start_date) & (daily_df['ActivityDate'] <= end_date)
week_df = daily_df.loc[mask]

# Extract day name for x-axis
week_df['DayName'] = week_df['ActivityDate'].dt.day_name()

# Sort days in proper order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week_df['DayName'] = pd.Categorical(week_df['DayName'], categories=day_order, ordered=True)
week_df = week_df.sort_values('DayName')

# Plot total steps per day for the week
plt.figure(figsize=(9, 5))
plt.bar(week_df['DayName'], week_df['TotalSteps'], color='cornflowerblue')

plt.title(f'Total Steps Per Day ({start_date.date()} to {end_date.date()})')
plt.xlabel('Day of Week')
plt.ylabel('Total Steps')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


##Zone_Minute/week

In [ ]:

# Copy and prepare data
daily_df = user_tables['Daily Activity'].copy()
daily_df['ActivityDate'] = pd.to_datetime(daily_df['ActivityDate'])

# Define week range (Monday to Sunday)
start_date = pd.to_datetime('2016-03-26')  # Monday
end_date = pd.to_datetime('2016-04-01')    # Sunday

# Filter data for the week
mask = (daily_df['ActivityDate'] >= start_date) & (daily_df['ActivityDate'] <= end_date)
week_df = daily_df.loc[mask]

# Calculate zone minutes per day
week_df['ZoneMinutes'] = (week_df['VeryActiveMinutes'] * 2) + week_df['FairlyActiveMinutes']

# Get day names for x-axis
week_df['DayName'] = week_df['ActivityDate'].dt.day_name()

# Sort by weekday order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week_df['DayName'] = pd.Categorical(week_df['DayName'], categories=day_order, ordered=True)
week_df = week_df.sort_values('DayName')

week_df['DayDateLabel'] = week_df.apply(
    lambda row: f"{row['DayName']}\n{row['ActivityDate'].strftime('%Y-%m-%d')}", axis=1
)
# Plot zone minutes per day
plt.figure(figsize=(10, 6))
bars = plt.bar(week_df['DayDateLabel'], week_df['ZoneMinutes'], color='mediumseagreen')

# Add value labels on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height)}',
             ha='center', va='bottom', fontsize=10)

plt.title(f'Zone Minutes Per Day ({start_date.date()} to {end_date.date()})')
plt.xlabel('Day and Date')
plt.ylabel('Zone Minutes')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()





## HeartRate/Week

In [ ]:
# Copy heart rate data
heartrate_df = user_tables['Heart Rate'].copy()

# Convert 'Time' to datetime
heartrate_df['Time'] = pd.to_datetime(heartrate_df['Time'])

# Extract date from datetime
heartrate_df['Date'] = heartrate_df['Time'].dt.date

# Define week range (Monday to Sunday)
start_date = pd.to_datetime('2016-04-01')  # Monday
end_date = pd.to_datetime('2016-04-07')    # Sunday

# Filter data for the week
mask = (heartrate_df['Date'] >= start_date.date()) & (heartrate_df['Date'] <= end_date.date())
week_df = heartrate_df.loc[mask].copy() # Add .copy() to avoid SettingWithCopyWarning later

# Check if the filtered DataFrame is empty
if not week_df.empty:
    # Group by Date and calculate average heart rate
    avg_hr_by_date = week_df.groupby('Date')['Value'].mean()

    # Sort by date (just to be safe)
    avg_hr_by_date = avg_hr_by_date.sort_index()

    # Plot average heart rate per day for the week
    plt.figure(figsize=(10, 6))
    avg_hr_by_date.plot(kind='bar', color='tomato')

    plt.title(f'Average Heart Rate per Day ({start_date.date()} to {end_date.date()}) for User {user_id}')
    plt.xlabel('Date')
    plt.ylabel('Avg Heart Rate (bpm)')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()
else:
    # Print a message if no data is found for the specified week
    print(f"No heart rate data available for the week from {start_date.date()} to {end_date.date()} for User {user_id}")